# Naive

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training import Naive
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

2024-01-09 23:54:19.075216: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jesusoyanedel/.pyenv/versions/3.10.11/envs/tt2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 50
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/naive.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

## Evaluation metrics definition

In [8]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),

    loggers=[InteractiveLogger()]
)

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [9]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = Naive(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [10]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/50 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:10<00:00, 46.44it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:10<00:00, 46.18it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:10<00:00, 45.83it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:10<00:00, 43.63it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:10<00:00, 43.83it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 47.16it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9108
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 47.16it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1377
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 47.65it/s]
> Eval on

  2%|▏         | 1/50 [02:24<1:57:58, 144.46s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1157
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = 0.0000
	StreamForgetting/eval_phase/test_stream = 0.0000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1333
-- >> Start of training phase << --
100%|██████████| 469/469 [00:11<00:00, 40.09it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:11<00:00, 40.77it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:11<00:00, 40.37it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:11<00:00, 40.18it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:11<00:00, 40.97it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 42.72it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0157
	ExperienceForgetting/eval_phase/test_stream/

  4%|▍         | 2/50 [04:53<1:57:53, 147.37s/it]

...

 96%|█████████▌| 48/50 [1:58:18<05:06, 153.24s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.0846
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.6254
	StreamForgetting/eval_phase/test_stream = 0.6254
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.3183
-- >> Start of training phase << --
100%|██████████| 469/469 [00:11<00:00, 39.76it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:11<00:00, 39.79it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:11<00:00, 39.83it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:11<00:00, 39.87it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:11<00:00, 39.79it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 42.22it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.7312
	ExperienceForgetting/eval_phase/test_stream

 98%|█████████▊| 49/50 [2:00:51<02:33, 153.22s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1379
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.6332
	StreamForgetting/eval_phase/test_stream = 0.6332
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.3159
-- >> Start of training phase << --
100%|██████████| 469/469 [00:11<00:00, 39.84it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:11<00:00, 39.81it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:11<00:00, 40.31it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:11<00:00, 39.98it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:11<00:00, 39.76it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:01<00:00, 42.60it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.7796
	ExperienceForgetting/eval_phase/test_stream

100%|██████████| 50/50 [2:03:24<00:00, 148.09s/it]


# Get metrics

In [11]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

# Store metrics

In [12]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))